In [ ]:
import pandas as pd
import numpy as np
import pprint as pprint
import re
import sqlalchemy as sqla


### Loadind data from csv files into datafreames


In [ ]:
# Restaurants
dfaccepts=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozaccepts.csv',encoding='ascii')
dfcuisine=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozcuisine.csv',encoding='utf8')
dfhours4=pd.read_csv(filepath_or_buffer='Restaurant-data/chefmozhours4.csv',encoding='utf8')
dfparkingdata2=pd.read_csv('Restaurant-data/chefmozparking.csv')
dfgeoplaces=pd.read_csv(filepath_or_buffer='Restaurant-data/geoplaces2.csv',encoding='latin-1')
# # consumers
dfusercuisine=pd.read_csv('Restaurant-data/usercuisine.csv')
dfuserpayment=pd.read_csv('Restaurant-data/userpayment.csv')
dfuserprofile=pd.read_csv('Restaurant-data/userprofile.csv')
# user item rating
dfrating=pd.read_csv('Restaurant-data/rating_final.csv')


###  Data cleaning

In [ ]:
# To perform general cleaning on all datasets and inspecting.
# removes dublicate and null rows
def cleandf(data):
    data.replace('?',np.NaN,inplace=True)
    print('no. of DUBLICATES', data.duplicated().sum(axis=0))
    print('NULL ROWS',data.isnull().sum(axis=0))
    data.drop_duplicates(inplace = True)
    data.dropna(inplace=True)
#     adult['sex'].map(lambda x: x.lower()).head()
    print(data.shape)
    return 0
cleandf(dfaccepts)
cleandf(dfgeoplaces)
cleandf(dfparkingdata2)
cleandf(dfhours4)
cleandf(dfcuisine)
cleandf(dfusercuisine)
cleandf(dfuserpayment)
cleandf(dfuserprofile)
cleandf(dfrating)




In [ ]:
# data cleaning
# replace all ? with nan
def cleandfgeoplaces():
    dfgeoplaces.isnull().sum(axis=0)
    # Drop null collunms bellow a given threshhod here 74. Removes zip,url and fax
    dfgeoplaces.dropna(thresh=74,axis=1,inplace=True)
    # print (dfgeoplaces.isnull().sum(axis=0))
    # print('Null rows ', dfgeoplaces.isnull().sum(axis=1))
    stripped=dfgeoplaces.select_dtypes(include='object').apply(lambda x: x.str.strip())
#     stripped=dfgeoplaces.select_dtypes(include='object').apply(lambda x: np.empty if x=='?' else x)

    typofreeCity= stripped.city.map(lambda x : 
            'San Luis Potosi' if (x=='s.l.p.') |(x=='s.l.p')| (x=='san luis potos' )| (x=='slp') | (x=='san luis potosi')else
            'Ciudad Victoria' if (x=='Cd. Victoria') | (x=='victoria') | (x=='Cd Victoria') | (x=='Victoria') else x)
    stripped.city=typofreeCity
    typofreeState= stripped.city.map(lambda x : 
            'San Luis Potosi' if (x=='s.l.p.') |(x=='s.l.p')| (x=='san luis potos' )| (x=='slp') | (x=='san luis potosi')else
            'Ciudad Victoria' if (x=='Cd. Victoria') | (x=='victoria') | (x=='Cd Victoria') | (x=='Victoria') else x)
    stripped.state=typofreeState
    stripped.country.fillna('Mexico')
    return stripped
cleandfgeoplaces()
# dfgeoplaces.address.map(lambda x: 'ooooooo' if x=='NaN' else x ) 

# pd.unique(better)

# dfgeoplaces.axes

# stripped[['city','state','country']][(stripped.city=='victoria')  | (stripped.city=='Cd Victoria')]


# pd.unique(dfgeoplaces.state)
# pd.unique(typofreeState)
# dfgeoplaces.country
# np.empty

In [ ]:
def dfhours4clean():
#    removing multiple ranges data fro hours collumn
    dfhours4.hours=dfhours4.hours.map(lambda x:re.split(';',x)[0])
#     dfhours4.hours.
    fromto = pd.DataFrame(dfhours4.hours.str.split('-',1).tolist(), columns = ['fromhour','tohour'])
    hourRange=pd.concat([dfhours4, fromto], axis=1, sort=False)
    return hourRange 
dfhours4clean()

### concartenating dataphrames

In [ ]:
# 3.1. Concatenate all cleaned data frames from ‘Restaurant’ category based on the common column, to create a large data frame and name it as Restaurants.
dfrestaurants=dfgeoplaces.merge(dfcuisine,how='left',on='placeID').merge(dfhours41,how='left',on='placeID').merge(dfaccepts,how='left',on='placeID').merge(dfparkingdata2,how='left',on='placeID')


In [ ]:
# 3.2. Concatenate all cleaned data frames from ‘Customer’ categories based on the common column, to create a large data frame and name it as Customers.
dfcustomers=dfuserprofile.merge(dfuserpayment,on='userID',how='left').merge(dfusercuisine, on='userID' , how='left')
dfcustomers.shape

### Creating Database and database tables



In [ ]:
# 3.3. Import the required package and create an engine for the sqlalchemy application:
engine = sqla.create_engine('sqlite:///data_science.db',echo=False)

In [ ]:
# 3.4. Create three SQL tables from three data frames: ‘Restaurants’, ‘Customers’
# which are created above, and the database from ‘rate’ category. The required

dfrestaurants.to_sql(name='restaurants', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
dfcustomers.to_sql(name='customers', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)
dfrating.to_sql(name='rating', con=engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)


### Queries

#### Assignment3  Part 1:

In [ ]:
# 2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?
# select where name is tamaulipas
engine.execute("SELECT distinct name FROM restaurants where state='tamaulipas'").fetchall()


In [ ]:
# 2.1.4. How many different customers used public transport for going to therestaurants?
engine.execute("select  count(userID) from (SELECT  distinct userID FROM customers where transport='public')").fetchall()


In [ ]:
# 2.1.5. What is the least popular payment method among customers?
engine.execute(" select upayment,MIN(paymentcount) from (select upayment,count(upayment)as paymentcount from ( SELECT distinct upayment,userID FROM customers) where upayment!='None' group by upayment)").fetchall()#=======================get the least


In [ ]:
# 2.1.6. How many (different) restaurants work until 19:00 in the evenings?
engine.execute(''' select count(*) from (SELECT distinct placeID from restaurants  where tohour="19:00") ''').fetchall()


In [ ]:
# 2.1.7. Which type of food (rcuisine) is served most among the restaurants?
engine.execute("Select Rcuisine,MAX(cuisineCoint) from (SELECT count(Rcuisine) as cuisineCoint,Rcuisine from (SELECT distinct placeID,Rcuisine FROM restaurants) group by Rcuisine)").fetchall()

In [ ]:
# 2.1.8. What is the percentage of customers who were born between 1980 and 1990?
engine.execute('''select (select count(*) from(SELECT distinct userID FROM customers where birth_year>=1980 and birth_year<=1990) as filteredCustomers) *100 /
                 (select count(userID) from (SELECT distinct userID FROM customers))''').fetchall()



In [ ]:
# 2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?

engine.execute(''' SELECT (SELECT count(*) from ( SELECT distinct userID FROM customers where transport="on foot" and budget="medium")) *100 /
                  (SELECT count(*) from ( SELECT distinct userID FROM customers where budget="medium") )''').fetchall()


### Assignment3- Part 2:

In [ ]:
# 2.2.1. What are the names of restaurants that do not have public parking lots?
engine.execute("SELECT distinct name FROM restaurants where parking_lot!='public'").fetchall()


In [ ]:
# 2.2.2. What are the addresses of restaurants which only accept ‘cash’?
engine.execute(" SELECT address from (SELECT  distinct placeid,address FROM restaurants where Rpayment='cash') ").fetchall()



In [ ]:
# 2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

engine.execute("SELECT distinct city from restaurants where placeID in (SELECT distinct placeID FROM restaurants where Rcuisine='Fast_Food')").fetchall()



In [ ]:
# 2.2.4. What is the most common ‘rating’ among customers with family?
engine.execute("SELECT rating from (Select MAX(ratingcount),rating from (SELECT count(rating) as ratingcount,rating from rating where userID in (SELECT distinct customers.userID  FROM customers where ambience='family') group by rating))").fetchall()


In [ ]:
# 2.2.5. What types of food (‘rcuisine’) received the highest rank from people with‘low’ budget?

engine.execute(''' SELECT MAX(totalRating),Rcuisine from 
                   (select Rcuisine,sum(food_rating) as totalRating from 
                   (Select * from (SELECT distinct userID as userID1, Rcuisine from customers)  left join 
                   rating on rating.userID=userId1 where 
                   userID1 in (SELECT distinct userID from customers where budget='low'))
                   group by Rcuisine)''').fetchall()




In [ ]:
# 2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants which just served ‘Wine-Beer’?

# three layer nested select where the firsr select finds avverage
# the middle select finds the customer who are social drinkers and the third restourants which serve alchohal


engine.execute(" SELECT AVG(service_rating) from rating where userID in (SELECT distinct userID from  customers where drink_level='social drinker') and placeID in (SELECT distinct placeID from  restaurants where alcohol='Wine-Beer')").fetchall()


In [ ]:
# 2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an open area?


engine.execute('''select count(userID) from (SELECT  distinct userID from customers where smoker="true" and userID in 
                 (SELECT userID from rating where service_rating=0 and placeID in 
                 (SELECT distinct placeID from restaurants where area!="open")))''').fetchall()
